In [1]:
import numpy as np
import os
import pandas as pd
from pandas_profiling import ProfileReport
import semopy

In [2]:
df_scores = pd.read_csv(os.path.join('data', '12_subject_scores.csv'), sep='\t')

In [3]:
display(df_scores.head())

,国語,英語,数学,物理,化学,生物,地学,世界史,日本史,経済,地理,倫理
0,-0.394296,-0.863552,0.299784,0.052264,-0.193747,1.762086,-0.458840,-0.889665,-0.163093,-0.939310,-0.149300,-2.286900
1,-1.062188,-0.061273,-0.572619,-0.544283,-0.305108,1.988985,-0.166363,0.610966,-0.951714,1.390854,-0.226003,-0.031098
2,-0.669470,-0.751437,0.574793,-0.379771,-0.539259,-0.623499,-1.034320,0.393436,1.439274,2.218160,0.716289,-0.462702
3,0.776026,1.166164,-0.986413,-0.723857,-0.091960,-0.150870,-0.047286,-2.427921,0.433994,-1.270779,0.323981,1.406393
4,-2.212173,-0.571762,-0.842556,-0.849638,-0.280362,-1.089733,-0.836957,-0.812882,-1.702264,-0.365675,-0.113076,-2.497812


In [4]:
# Define
mod = """\
lang =~ 国語 + 英語 + 倫理
math =~ 数学 + 物理 + 化学
memory =~ 生物 + 地学 + 日本史 + 世界史 + 地理 + 経済
math ~ lang
memory ~ lang
math ~~ memory
"""

model = semopy.Model(mod)

In [5]:
# load data
model.load_dataset(df_scores)

In [8]:
opt = semopy.Optimizer(model)
objective_function_value = opt.optimize()
objective_function_value

AttributeError: module 'semopy' has no attribute 'Optimizer'

In [9]:
df_result = semopy.inspect(opt)

df_result

AttributeError: module 'semopy' has no attribute 'inspect'

In [10]:
df_result = pd.conct([df_result, df_trivial_result], axis=0, ignore_index=True)

df_result

AttributeError: module 'pandas' has no attribute 'conct'

In [11]:
df_result_measurement = df_result[df_result['op'] == '=~']
df_result_variance = df_result[df_result['op'] == '~~']
dropping_index = df_result_variance[(df_result_variance['lval'] == 'math') & (df_result_variance['rval'] == 'memory')].index.values[0]
df_result_math_memory = df_result.iloc[dropping_index, :]
df_result_variance = df_result_variance.drop(dropping_index, axis=0)

print('df_result_structural')
display(df_result_structural)

print('df_result_measurement')
display(df_result_measurement)

print('df_result_math_memory')
display(df_result_math_memory)

print('df_result_variance')
display(df_result_variance)

NameError: name 'df_result' is not defined

In [12]:
# step 1 variance of lang
df_result_variance['variable'] = df_result_variance['lval']
df_result_variance = df_result_variance.set_index('variable')
df_result_variance['variance'] = np.nan

lang_var = df_result_variance.loc['lang', 'Value']
df_result_variance.loc['lang', 'variance'] = lang_var

df_result_variance

NameError: name 'df_result_variance' is not defined

In [13]:
# step 2 variance of math, memory

def compute_var_of_math_memory(target):
    coef = df_result_structural[df_result_structural['lval'] == target]['Value'].values[0]
    var = df_result_variance[df_result_variance['lval'] == target]['Value'].values[0] + lang_var * coef**2
    return var

var_math = compute_var_of_math_memory('math')
var_memory = compute_var_of_math_memory('memory')

df_result_variance.loc['math', 'variance'] = var_math
df_result_variance.loc['memory', 'variance'] = var_memory

df_result_variance


NameError: name 'df_result_structural' is not defined

In [14]:
# step 3 variance of manifest variables

map_latent_manifest = {
    '国語': 'lang',
    '英語': 'lang',
    '倫理': 'lang',
    '数学': 'math',
    '物理': 'math',
    '化学': 'math',
    '生物': 'memory',
    '地学': 'memory',
    '日本史': 'memory',
    '世界史': 'memory',
    '地理': 'memory',
    '経済': 'memory',
}

def compute_var_of_manifest_var(target):
    latent = map_latent_manifest[target]
    latent_var = df_result_variance.loc[latent, 'variance']
    coef = df_result_measurement[(df_result_measurement['rval'] == target)]['Value'].values[0]
    var = df_result_variance[df_result_variance['lval'] == target]['Value'].values[0] + latent_var * coef**2
    return var

for key in map_latent_manifest.keys():
    df_result_variance.loc[key, 'variance'] = compute_var_of_manifest_var(key)

df_result_variance


NameError: name 'df_result_variance' is not defined

In [15]:
# step 4 std coef of structural part

df_result_structural = df_result_structural.merge(df_result_variance[['variance']], how='left', left_on='lval', right_index=True).rename(columns={'variance': 'lvariance'})
df_result_structural = df_result_structural.merge(df_result_variance[['variance']], how='left', left_on='rval', right_index=True).rename(columns={'variance': 'rvariance'})
df_result_structural['std_coef'] = df_result_structural['Value'] / np.sqrt(df_result_structural['lvariance']) * np.sqrt(df_result_structural['rvariance'])

display(df_result_structural)

df_result_math_memory = pd.DataFrame(df_result_math_memory).T
df_result_math_memory = df_result_math_memory.merge(df_result_variance[['variance']], how='left', left_on='lval', right_index=True).rename(columns={'variance': 'lvariance'})
df_result_math_memory = df_result_math_memory.merge(df_result_variance[['variance']], how='left', left_on='rval', right_index=True).rename(columns={'variance': 'rvariance'})
df_result_math_memory['std_coef'] = df_result_math_memory['Value'] / np.sqrt(df_result_math_memory['lvariance']) / np.sqrt(df_result_math_memory['rvariance'])

display(df_result_math_memory)


NameError: name 'df_result_structural' is not defined

In [16]:

# step 5 std coef of measurement part

df_result_measurement = df_result_measurement.merge(df_result_variance[['variance']], how='left', left_on='lval', right_index=True).rename(columns={'variance': 'lvariance'})
df_result_measurement = df_result_measurement.merge(df_result_variance[['variance']], how='left', left_on='rval', right_index=True).rename(columns={'variance': 'rvariance'})
df_result_measurement['std_coef'] = df_result_measurement['Value'] * np.sqrt(df_result_measurement['lvariance']) / np.sqrt(df_result_measurement['rvariance'])

df_result_measurement

NameError: name 'df_result_measurement' is not defined

In [17]:

# step 6 std coef of manifest variable part

df_result_variance = df_result_variance.rename(columns={'variance': 'lvariance'})
df_result_variance['rvariance'] = df_result_variance['lvariance']

df_result_variance['std_coef'] = np.sqrt(df_result_variance['Value'] / df_result_variance['lvariance'])

df_result_variance

NameError: name 'df_result_variance' is not defined